### Tersor....Board....ing 



#### Watching the [introduction video](https://www.youtube.com/watch?v=eBbEDRsCmv4&t=1105s), stop focusing on Dandelion Mané's fancy nail polishing  (that is the purple elephant with mint polka dot) [Demo code](https://github.com/decentralion/tf-dev-summit-tensorboard-tutorial)

```

```
If (Tensorboard == useful): 
    Install tensorflow https://www.tensorflow.org/install   
else 
    Close this notebook

TODO 
    create the input data
    

#### start the tensorboard : 

    /Prometheus/src$ tensorboard --logdir=./logs_ann/
....

2018-08-02 07:44:08.927850: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA

TensorBoard 1.8.0 at __http://*****:6006__ (Press CTRL+C to quit)


Tips: if you open the 6006 url link but find nothing, calm down, double check the path. Wrong folder will give you empty board 


### Wrong code could too, so the code begins here

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from os import listdir
from tensorflow.python.client import timeline

import calendar
import time
import arrow
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU,Input
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.losses import mean_squared_error

In [2]:
# import data and tailor it for use
import pickle
with open("data", "rb") as f:
    df_X = pickle.load(f)
    df_Y = pickle.load(f)

shift_iter = 24
X = df_X.values[shift_iter + 1:-shift_iter - 1]
Y = df_Y.values[shift_iter + 1:-shift_iter - 1]
print(X.shape)
print(Y.shape)
print(list(df_X),list(df_Y))

(8710, 77)
(8710, 24)
['Utetemperatur T+1', 'Energianvändning T+1', 'Utetemperatur T+2', 'Energianvändning T+2', 'Utetemperatur T+3', 'Energianvändning T+3', 'Utetemperatur T+4', 'Energianvändning T+4', 'Utetemperatur T+5', 'Energianvändning T+5', 'Utetemperatur T+6', 'Energianvändning T+6', 'Utetemperatur T+7', 'Energianvändning T+7', 'Utetemperatur T+8', 'Energianvändning T+8', 'Utetemperatur T+9', 'Energianvändning T+9', 'Utetemperatur T+10', 'Energianvändning T+10', 'Utetemperatur T+11', 'Energianvändning T+11', 'Utetemperatur T+12', 'Energianvändning T+12', 'Utetemperatur T+13', 'Energianvändning T+13', 'Utetemperatur T+14', 'Energianvändning T+14', 'Utetemperatur T+15', 'Energianvändning T+15', 'Utetemperatur T+16', 'Energianvändning T+16', 'Utetemperatur T+17', 'Energianvändning T+17', 'Utetemperatur T+18', 'Energianvändning T+18', 'Utetemperatur T+19', 'Energianvändning T+19', 'Utetemperatur T+20', 'Energianvändning T+20', 'Utetemperatur T+21', 'Energianvändning T+21', 'Utetemp

In [3]:
ratio = 0.75  # for spliting train - test set
num_train = round(X.shape[0] * ratio)
x_train = X[:num_train]
y_train = Y[:num_train]
x_test = X[num_train:]
y_test = Y[num_train:]

x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

num_x_signals = X.shape[1]
num_y_signals = Y.shape[1]

In [4]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)

            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx + sequence_length]
            y_batch[i] = y_train_scaled[idx:idx + sequence_length]

        yield (x_batch, y_batch)

#### create a model from tensorflow using API : Keras 

In [5]:
### TODO change the simple_value into histogram DONE

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs_for_tensorboard', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super().__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super().set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        print("\n","val_logs", "\n",val_logs)
        for name, value in val_logs.items():
            
#             # histogram in Tensorboard
#             values = np.array(value)
#             # Create histogram using numpy
#             counts, bin_edges = np.histogram(values, bins=1000)
#             # Fill fields of histogram proto
#             hist = tf.HistogramProto()
#             hist.min = float(np.min(values))
#             hist.max = float(np.max(values))
#             hist.num = int(np.prod(values.shape))
#             hist.sum = float(np.sum(values))
#             hist.sum_squares = float(np.sum(values ** 2))
#             bin_edges = bin_edges[1:]

#             # Add bin edges and counts
#             for edge in bin_edges:
#                 hist.bucket_limit.append(edge)
#             for c in counts:
#                 hist.bucket.append(c)
#             summary = tf.Summary(value=[tf.Summary.Value(tag=name, histo=hist)])
            
#             self.val_writer.add_summary(summary, epoch)
            # scalars in tensorboard
            summary = tf.Summary(value=[tf.Summary.Value
                              (tag=name, simple_value=value)
                             ]
                      )
            
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super().on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super().on_train_end(logs)
        self.val_writer.close()

In [6]:
model = Sequential()

model.add(Dense(num_x_signals *2 ,
              activation='hard_sigmoid',
              name = "hidden",
              input_shape=(None, num_x_signals,)))

model.add(Dense(num_y_signals, 
                activation='linear',
                name = "output"))

model.compile(loss=mean_squared_error,
              optimizer=Adam(lr=1e-2),
              metrics=['accuracy','mae','mean_squared_error','mape'],
              options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE),
              run_metadata= tf.RunMetadata(), )

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, None, 154)         12012     
_________________________________________________________________
output (Dense)               (None, None, 24)          3720      
Total params: 15,732
Trainable params: 15,732
Non-trainable params: 0
_________________________________________________________________


In [7]:
%%time

generator = batch_generator(batch_size=200, sequence_length=168)
log_dir="./logs_tensorboard_demo/{}".format(arrow.now().format('YYYY-MM-DD-HH-MM-SS'))
print(log_dir)

model.fit_generator(generator=generator,
                    epochs=10,
                    verbose = 1,
                    steps_per_epoch=80,
                    validation_data=validation_data,
                    callbacks=[TrainValTensorBoard(
                           log_dir,
                           histogram_freq = 1,
                           write_graph = True, # visualize the graph in TensorBoard
                           write_images = True, #  write model weights to visualize as image in TensorBoard
                           write_grads = True, # visualize gradient histograms in TensorBoard
                                  )])

./logs_tensorboard_demo/2018-08-02-14-08-60
Epoch 1/10
79/80 [============================>.] - ETA: 0s - loss: 0.0397 - acc: 0.0642 - mean_absolute_error: 0.1187 - mean_squared_error: 0.0397 - mean_absolute_percentage_error: 31112.6070
 val_logs 
 {'loss': 0.002709698397666216, 'acc': 0.0555555559694767, 'mean_absolute_error': 0.04108614847064018, 'mean_squared_error': 0.002709698397666216, 'mean_absolute_percentage_error': 19.342060089111328}
80/80 [==============================] - 5s 63ms/step - loss: 0.0393 - acc: 0.0644 - mean_absolute_error: 0.1178 - mean_squared_error: 0.0393 - mean_absolute_percentage_error: 30965.1173 - val_loss: 0.0027 - val_acc: 0.0556 - val_mean_absolute_error: 0.0411 - val_mean_squared_error: 0.0027 - val_mean_absolute_percentage_error: 19.3421
Epoch 2/10
79/80 [============================>.] - ETA: 0s - loss: 0.0025 - acc: 0.0913 - mean_absolute_error: 0.0387 - mean_squared_error: 0.0025 - mean_absolute_percentage_error: 15331.1771
 val_logs 
 {'loss': 